In [4]:
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Define the dataset path
dataset_path = "C:/Users/user/anaconda3/envs/flower-ai-fixed/flowers-dataset"

# Check if the path exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"❌ Dataset path does not exist: {dataset_path}")
else:
    print(f"✅ Dataset path found: {dataset_path}")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit model input
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load datasets
train_dataset = ImageFolder(root=f"{dataset_path}/train", transform=transform)
val_dataset = ImageFolder(root=f"{dataset_path}/validation", transform=transform)
test_dataset = ImageFolder(root=f"{dataset_path}/test", transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check dataset details
print(f"✅ Train Dataset: {len(train_dataset)} images in {len(train_dataset.classes)} classes")
print(f"✅ Validation Dataset: {len(val_dataset)} images")import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer
criterion = torch.nn.CrossEntropyLoss()  # Loss function

print(f"✅ Test Dataset: {len(test_dataset)} images")
print(f"📁 Classes: {train_dataset.classes}")


✅ Dataset path found: C:/Users/user/anaconda3/envs/flower-ai-fixed/flowers-dataset
✅ Train Dataset: 1200 images in 3 classes
✅ Validation Dataset: 2532 images
✅ Test Dataset: 292 images
📁 Classes: ['fresh daisy', 'fresh rose', 'fresh tulip']


In [3]:
import torch
import torchvision.models as models
import torch.nn as nn

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {device}")

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Modify the final layer for 3 classes (daisy, rose, tulip)
num_classes = 3
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to device
model = model.to(device)

# Print model summary
print(model)


✅ Using Device: cpu


C:\Users\user\anaconda3\envs\flower-ai-fixed\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\anaconda3\envs\flower-ai-fixed\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models

# Define training parameters
num_epochs = 10  # Adjust as needed
learning_rate = 0.001
batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {device}")

# Load ResNet model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3 classes: daisy, rose, tulip
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    
print("✅ Training Complete!")


✅ Using Device: cpu


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import os

# ✅ Check for device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {device}")

# ✅ Define dataset paths
dataset_path = "C:/Users/user/anaconda3/envs/flower-ai-fixed/flowers-dataset"

# ✅ Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Load datasets
train_dataset = ImageFolder(root=os.path.join(dataset_path, "train"), transform=transform)
val_dataset = ImageFolder(root=os.path.join(dataset_path, "validation"), transform=transform)
test_dataset = ImageFolder(root=os.path.join(dataset_path, "test"), transform=transform)

# ✅ Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ✅ Load pre-trained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(512, 3)  # 3 flower classes
model = model.to(device)

# ✅ Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Training Loop
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    # ✅ Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    # ✅ Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_flower_model.pth")
        print("✅ Model Saved!")

# ✅ Testing
model.load_state_dict(torch.load("best_flower_model.pth"))
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"✅ Final Test Accuracy: {test_acc:.4f}")


In [ ]:
# Check dataset sizes
print(f"Train Dataset: {len(train_dataset)} images")
print(f"Validation Dataset: {len(val_dataset)} images")
print(f"Test Dataset: {len(test_dataset)} images")

# Check class names
print(f"Classes: {train_dataset.classes}")


In [2]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

# Define dataset path
data_dir = r"C:\Users\user\anaconda3\envs\flower-ai-fixed\flowers-dataset"

# Load datasets
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)

# Print to confirm
print(f"✅ Train Dataset Loaded: {len(train_dataset)} images")


✅ Train Dataset Loaded: 1200 images


In [3]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"✅ DataLoader Loaded: {len(train_loader)} batches")


✅ DataLoader Loaded: 38 batches


In [4]:
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Batch Size: {train_loader.batch_size}")
print(f"Total Batches: {len(train_loader)}")


Train Dataset Size: 1200
Batch Size: 32
Total Batches: 38


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the last layer to match your 3 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

# Move model to CPU (since you don’t have a GPU)
device = torch.device("cpu")
model.to(device)

print("✅ Model Loaded and Modified!")


✅ Model Loaded and Modified!


In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("✅ Loss Function & Optimizer Defined!")


✅ Loss Function & Optimizer Defined!


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(train_loader):.4f}")

print("✅ Training Completed!")


In [ ]:
print(f"Total Training Batches: {len(train_loader)}")


In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define transformations for data augmentation & normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root="C:/Users/user/anaconda3/envs/flower-ai-fixed/flowers-dataset/train", transform=transform)

# Define DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"✅ Train DataLoader Ready! Total Batches: {len(train_loader)}")


✅ Train DataLoader Ready! Total Batches: 38


In [5]:
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Total Training Batches: {len(train_loader)}")


Train Dataset Size: 1200
Total Training Batches: 38


In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device set to:", device)


✅ Device set to: cpu


In [9]:
import torchvision.models as models

# Load pre-trained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final layer to match your number of classes (3 classes: daisy, rose, tulip)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3)

# Move model to the correct device (CPU in your case)
model.to(device)

print("✅ Model defined and moved to device!")


✅ Model defined and moved to device!


In [10]:
print("Train Loader Exists:", 'train_loader' in globals())
print("Train Dataset Exists:", 'train_dataset' in globals())
print("Model Exists:", 'model' in globals())
print("Device:", device)


Train Loader Exists: True
Train Dataset Exists: True
Model Exists: True
Device: cpu


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple CNN model
class FlowerCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(FlowerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 64 * 64, num_classes)  # Adjust based on image size

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = self.fc1(x)
        return x

# Initialize the model
model = FlowerCNN(num_classes=3)

# Move model to CPU
device = torch.device("cpu")  
model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("✅ Model, optimizer, and loss function initialized successfully!")


✅ Model, optimizer, and loss function initialized successfully!


In [7]:
print("Train Dataset Exists:", 'train_dataset' in globals())


Train Dataset Exists: False


In [9]:
print("Train Dataset Exists:", 'train_dataset' in globals())


Train Dataset Exists: False


In [11]:
import os
print("Train Folder Exists:", os.path.exists("dataset/train"))
print("Folders inside 'dataset':", os.listdir("dataset") if os.path.exists("dataset") else "No dataset folder found")


Train Folder Exists: True
Folders inside 'dataset': ['test', 'train']


In [12]:
import os

train_path = "dataset/train"
print("Subfolders inside 'train':", os.listdir(train_path))


Subfolders inside 'train': []


In [14]:
import os

dataset_path = "dataset/train"

if os.path.exists(dataset_path):
    print("✅ Dataset folder found!")
    print("✅ Subdirectories inside 'train':", os.listdir(dataset_path))
else:
    print("❌ Dataset folder not found!")


✅ Dataset folder found!
✅ Subdirectories inside 'train': []


In [15]:
import os

dataset_path = "dataset/train"

# Function to rename images
def rename_images(folder):
    for idx, filename in enumerate(os.listdir(folder)):
        old_path = os.path.join(folder, filename)
        new_filename = f"image_{idx+1}.jpg"  # Renaming to image_1.jpg, image_2.jpg, etc.
        new_path = os.path.join(folder, new_filename)

        os.rename(old_path, new_path)
        print(f"Renamed: {old_path} ➝ {new_path}")

# Rename images in each flower class folder
for flower in ["daisy", "rose", "tulip"]:
    folder_path = os.path.join(dataset_path, flower)
    if os.path.exists(folder_path):
        rename_images(folder_path)
    else:
        print(f"❌ Folder not found: {folder_path}")

print("✅ Image renaming completed!")


❌ Folder not found: dataset/train\daisy
❌ Folder not found: dataset/train\rose
❌ Folder not found: dataset/train\tulip
✅ Image renaming completed!


In [16]:
import os

dataset_path = "dataset/train"

if os.path.exists(dataset_path):
    print("✅ Train folder exists.")
    print("📂 Subfolders inside 'dataset/train':", os.listdir(dataset_path))
else:
    print("❌ 'dataset/train' folder NOT found! Check your path.")


✅ Train folder exists.
📂 Subfolders inside 'dataset/train': []


In [17]:
import os

dataset_path = "dataset"

# Recursively search for flower class folders
for root, dirs, files in os.walk(dataset_path):
    print(f"📂 Found folders in '{root}': {dirs}")



📂 Found folders in 'dataset': ['test', 'train']
📂 Found folders in 'dataset\test': []
📂 Found folders in 'dataset\train': []


In [18]:
print("📂 Subfolders inside 'dataset/train':", os.listdir("dataset/train"))


📂 Subfolders inside 'dataset/train': []


In [21]:
import os

dataset_path = "dataset/train"

if os.path.exists(dataset_path):
    print("✅ Train folder exists.")
    print("📂 Subfolders inside 'dataset/train':", os.listdir(dataset_path))
else:
    print("❌ 'dataset/train' folder NOT found! Check your path.")


✅ Train folder exists.
📂 Subfolders inside 'dataset/train': []


In [22]:
import os

dataset_path = "dataset/train"

# Function to rename images inside each class folder
def rename_images(folder, prefix):
    for idx, filename in enumerate(os.listdir(folder)):
        old_path = os.path.join(folder, filename)
        
        # Ensure only image files are renamed
        if os.path.isfile(old_path) and filename.lower().endswith((".jpg", ".jpeg", ".png")):
            new_filename = f"{prefix}_{idx+1}.jpg"  # e.g., daisy_1.jpg, rose_1.jpg, tulip_1.jpg
            new_path = os.path.join(folder, new_filename)

            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} ➝ {new_path}")

# Rename images for each flower class
for flower in ["daisy", "rose", "tulip"]:
    folder_path = os.path.join(dataset_path, flower)
    if os.path.exists(folder_path):
        rename_images(folder_path, flower)  # Use folder name as prefix
    else:
        print(f"❌ Folder not found: {folder_path}")

print("✅ Image renaming completed!")


❌ Folder not found: dataset/train\daisy
❌ Folder not found: dataset/train\rose
❌ Folder not found: dataset/train\tulip
✅ Image renaming completed!


In [1]:
import os

dataset_path = "dataset/train"

# Function to rename images inside each class folder
def rename_images(folder, prefix):
    for idx, filename in enumerate(os.listdir(folder)):
        old_path = os.path.join(folder, filename)
        
        # Ensure only image files are renamed
        if os.path.isfile(old_path) and filename.lower().endswith((".jpg", ".jpeg", ".png")):
            new_filename = f"{prefix}_{idx+1}.jpg"  # e.g., daisy_1.jpg, rose_1.jpg, tulip_1.jpg
            new_path = os.path.join(folder, new_filename)

            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} ➝ {new_path}")

# Rename images for each flower class
for flower in ["daisy", "rose", "tulip"]:
    folder_path = os.path.join(dataset_path, flower)
    if os.path.exists(folder_path):
        rename_images(folder_path, flower)  # Use folder name as prefix
    else:
        print(f"❌ Folder not found: {folder_path}")

print("✅ Image renaming completed!")


❌ Folder not found: dataset/train\daisy
❌ Folder not found: dataset/train\rose
❌ Folder not found: dataset/train\tulip
✅ Image renaming completed!


In [2]:
import os

dataset_path = "dataset/train"

if os.path.exists(dataset_path):
    print("✅ 'train' folder exists.")
    print("📂 Subfolders inside 'dataset/train':", os.listdir(dataset_path))
else:
    print("❌ 'dataset/train' folder NOT found! Check your path.")


✅ 'train' folder exists.
📂 Subfolders inside 'dataset/train': []


In [3]:
import os

dataset_path = "dataset"

if os.path.exists(dataset_path):
    print("✅ 'dataset' folder exists.")
    print("📂 Subfolders inside 'dataset':", os.listdir(dataset_path))
else:
    print("❌ 'dataset' folder NOT found!")


✅ 'dataset' folder exists.
📂 Subfolders inside 'dataset': ['test', 'train']


In [5]:
import os

dataset_path = "dataset"

if os.path.exists(dataset_path):
    print("✅ 'dataset' folder exists.")
    print("📂 Subfolders inside 'dataset':", os.listdir(dataset_path))
else:
    print("❌ 'dataset' folder NOT found!")


✅ 'dataset' folder exists.
📂 Subfolders inside 'dataset': ['test', 'train']


In [8]:
import os

dataset_path = "dataset"

for root, dirs, files in os.walk(dataset_path):
    print(f"📂 Found in {root}: {dirs}")


📂 Found in dataset: ['test', 'train']
📂 Found in dataset\test: []
📂 Found in dataset\train: []


In [9]:
import os

dataset_path = "dataset"

for root, dirs, files in os.walk(dataset_path):
    print(f"📂 Found in {root}: {dirs}")


📂 Found in dataset: ['test', 'train']
📂 Found in dataset\test: []
📂 Found in dataset\train: []


In [10]:
import os

dataset_path = "dataset"

for root, dirs, files in os.walk(dataset_path):
    print(f"📂 Found in {root}: {dirs}")
    if files:
        print(f"📄 Files in {root}: {files}")


📂 Found in dataset: ['test', 'train']
📂 Found in dataset\test: []
📂 Found in dataset\train: []
